# Cardiovascular disease prediction

This code uses 2 algorithms to serve the purpose.
Further imporovement is to made.

In [ ]:
# import libraries
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt
import seaborn as sns  # plot

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# load data
df = pd.read_csv("/Users/riteshkumar/Downloads/ML projects/Cardiovascular Disease Prediction/cardio_train.csv", sep=";")
df.head()

In [ ]:
print(df.columns)


In [ ]:
# drop 'id' column 
df.drop(columns=['id'], errors='ignore', inplace=True)


In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# visualize cardio with gender
sns.countplot(x='cardio',data=df,hue='gender',palette='rainbow')

In [ ]:
# distribution wrt age
sns.boxplot(x='cardio',y='age',data=df)

In [ ]:
plt.figure(figsize=(14,6))
plt.subplot(1,2,1)
sns.boxplot(x='cardio',y='height',data=df,palette='winter')
plt.subplot(1,2,2)
sns.boxplot(x='cardio',y='weight',data=df,palette='summer')

In [ ]:
# correlations with target class
correlations = df.corr()['cardio'].drop('cardio')
print(correlations)

In [ ]:
def feat_select(threshold):
    abs_cor = correlations.abs()
    features = abs_cor[abs_cor > threshold].index.tolist()
    return features

In [ ]:
def model(mod,X_tr,X_te):
    mod.fit(X_tr,y_train)
    pred = mod.predict(X_te)
    print('Model score = ',mod.score(X_te,y_test)*100,'%')

In [ ]:
# split data
msk = np.random.rand(len(df))<0.85
df_train_test = df[msk]
df_val = df[~msk]

X = df_train_test.drop('cardio',axis=1)
y = df_train_test['cardio']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=70)

In [ ]:
# for logistic regression
lr = LogisticRegression()

In [ ]:
threshold = [0.001,0.002,0.005,0.01,0.05,0.1]
for i in threshold:
    print('\n',i)
    feature_i = feat_select(i)
    X_train_i = X_train[feature_i]
    X_test_i = X_test[feature_i]
    model(lr,X_train_i,X_test_i)

In [ ]:
scale = StandardScaler()
scale.fit(X_train)
X_train_scaled = scale.transform(X_train)
X_train_ = pd.DataFrame(X_train_scaled,columns=df.columns[:-1])

In [ ]:
scale.fit(X_test)
X_test_scaled = scale.transform(X_test)
X_test_ = pd.DataFrame(X_test_scaled,columns=df.columns[:-1])

In [ ]:
# final feature selection with threshold 0.05
feat_final = feat_select(0.05)
print(feat_final)

In [ ]:
# scaling the val data as well
X_train = X_train_[feat_final]
X_val = np.asanyarray(df_val[feat_final])
y_val = np.asanyarray(df_val['cardio'])

scale.fit(X_val)
X_val_scaled = scale.transform(X_val)
X_val_ = pd.DataFrame(X_val_scaled,columns=df_val[feat_final].columns)

In [ ]:
# knn with k=15
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train,y_train)
pred = knn.predict(X_val_)

In [ ]:
# reports
print('Confusion Matrix =\n',confusion_matrix(y_val,pred))
print('\n',classification_report(y_val,pred))

In [ ]:
# Logistic regression
lr.fit(X_train,y_train)
pred = lr.predict(X_val_)

In [ ]:
# reports
print('Confusion Matrix =\n',confusion_matrix(y_val,pred))
print('\n',classification_report(y_val,pred))

In [ ]:
import pickle

with open("logistic_regression_model.pkl", "wb") as f:
    pickle.dump(lr, f)

with open("knn_model.pkl", "wb") as f:
    pickle.dump(knn, f)

print("Models saved successfully!")

Both give more or less the similar results. I am planning to work on this project to improve performance.